To execute this notebook, you need to either
 - Download a pre-trained model
 - Train an example model by excecuting the model_training_tutorial.ipynb notebook

In this tutorial we will learn to:
- Load a previously trained model
- Extract DeepPrint features from fingerprint images
- Evaluate the performance of the extracted fixed-length representations

## Embedding extraction

After training the model, we can extract the DeepPrint features for the fingerprint images. This is done by calling the `extract` method of the `DeepPrintExtractor` class.

In [1]:
import os

from flx.extractor.fixed_length_extractor import get_DeepPrint_Tex, get_DeepPrint_TexMinu, DeepPrintExtractor

# Dimension and number of training subjects must be known to load the pre-trained model

# To load the pre-trained model parameters use num_training_subjects=8000
extractor: DeepPrintExtractor = get_DeepPrint_TexMinu(num_training_subjects=140, num_dims=512)

# To load the pre-trained model parameters use
# MODEL_DIR: str = os.path.abspath("/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/DeepPrint_TexMinu_512") # Path to the directory 
MODEL_DIR: str = os.path.abspath("/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/models/FVC2006_DB3A_model")
# containing the model parameters
extractor.load_best_model(MODEL_DIR)

Loaded best model from /home/mt0/22CS60R42/fixed-length-fingerprint-extractors/models/FVC2006_DB3A_model/best_model.pyt


/home/mt0/22CS60R42/.anaconda3/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Now we need to specify the dataset, for which we want to extract the embeddings

In [2]:
import os

from flx.data.dataset import *
from flx.data.image_loader import SFingeLoader
from flx.data.transformed_image_loader import TransformedImageLoader
from flx.image_processing.binarization import LazilyAllocatedBinarizer
from flx.data.image_helpers import pad_and_resize_to_deepprint_input_size

# NOTE: If this does not work, enter the absolute path to the notebooks/example-dataset directory here! 
DATASET_PATH: str = os.path.abspath("/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/example-dataset-roi-images")

# We will use the SFingeLoader to load the images from the dataset
image_loader = TransformedImageLoader(
        images=SFingeLoader(DATASET_PATH),
        poses=None,
        transforms=[
            LazilyAllocatedBinarizer(5.0),
            pad_and_resize_to_deepprint_input_size,
        ],
    )

image_dataset: Dataset = Dataset(image_loader, image_loader.ids)

# The second value is for the minutiae branch, which we do not have in this example
texture_embeddings, minutia_embeddings = extractor.extract(image_dataset)

Created IdentifierSet with 6 subjects and a total of 60 samples.


100%|██████████| 2/2 [00:27<00:00, 13.91s/it]


In [3]:
from flx.benchmarks.matchers import CosineSimilarityMatcher
from flx.data.embedding_loader import EmbeddingLoader

# We concatenate texture and minutia embedding vectors
embeddings = EmbeddingLoader.combine(texture_embeddings, minutia_embeddings)


Created IdentifierSet with 6 subjects and a total of 60 samples.
Created IdentifierSet with 6 subjects and a total of 60 samples.
Created IdentifierSet with 6 subjects and a total of 60 samples.


In [10]:
print(embeddings._array);

[[-0.0067844  -0.01064628  0.00425365 ... -0.00924355 -0.0037603
   0.05010139]
 [-0.0067844  -0.01064628  0.00425365 ... -0.00924355 -0.0037603
   0.05010139]
 [-0.0067844  -0.01064628  0.00425365 ... -0.00924355 -0.0037603
   0.05010139]
 ...
 [-0.0067844  -0.01064628  0.00425365 ... -0.00924355 -0.0037603
   0.05010139]
 [-0.0067844  -0.01064628  0.00425365 ... -0.00924355 -0.0037603
   0.05010139]
 [-0.0067844  -0.01064628  0.00425365 ... -0.00924355 -0.0037603
   0.05010139]]


Waking algo for core to binary using gray code

In [4]:
import os

def gray_code(n):
    return n ^ (n >> 1)

def convert_to_gray_code(x, y):
    gray_x = gray_code(int(x))
    gray_y = gray_code(int(y))
    return bin(gray_x)[2:], bin(gray_y)[2:]

def pad_binary(binary, length):
    return binary.zfill(length)

def process_txt_file(txt_file, output_folder):
    with open(txt_file, 'r') as f:
        lines = f.readlines()
        x, y = map(float, lines[0].split())
        
        gray_x, gray_y = convert_to_gray_code(x, y)
        
        concatenated_binary = gray_x + gray_y
        
        max_length = len(concatenated_binary)
        
        padded_binary = pad_binary(concatenated_binary, max_length)
        
        output_file = os.path.join(output_folder, os.path.splitext(os.path.basename(txt_file))[0] + '_result.txt')
        with open(output_file, 'w') as out_f:
            out_f.write(padded_binary)

def process_folder(input_folder, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.endswith('.txt'):
            process_txt_file(os.path.join(input_folder, filename), output_folder)

# Specify input and output folders
input_folder = '/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/walking_core_points'
output_folder = '/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/walking_binary_core'

# Process all .txt files in the input folder
process_folder(input_folder, output_folder)


For Fingerflow convert core to binary using xor 


In [5]:
# import os

# def convert_to_binary(x, y):
#     binary_x = bin(int(x))[2:]
#     binary_y = bin(int(y))[2:]
#     return binary_x, binary_y

# def pad_binary(binary, length):
#     return binary.zfill(length)

# def xor_binary(binary1, binary2):
#     return bin(int(binary1, 2) ^ int(binary2, 2))[2:]

# def process_txt_file(txt_file):
#     with open(txt_file, 'r') as f:
#         lines = f.readlines()
#         x1, y1 = map(float, lines[0].split())
#         x2, y2 = map(float, lines[1].split())
        
#         binary_x1, binary_y1 = convert_to_binary(x1, y1)
#         binary_x2, binary_y2 = convert_to_binary(x2, y2)
        
#         s1 = binary_x1 + binary_y1
#         s2 = binary_x2 + binary_y2
        
#         max_length = max(len(s1), len(s2))
#         s1 = pad_binary(s1, max_length)
#         s2 = pad_binary(s2, max_length)
        
#         result = xor_binary(s1, s2)
        
#         output_file = os.path.splitext(txt_file)[0] + '_result.txt'
#         with open(output_file, 'w') as out_f:
#             out_f.write(result)

# def process_folder(input_folder):
#     for filename in os.listdir(input_folder):
#         if filename.endswith('.txt'):
#             process_txt_file(os.path.join(input_folder, filename))

# # Specify input folder containing .txt files
# input_folder = '/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/fixedlength-core_points2'

# # Process all .txt files in the folder
# process_folder(input_folder)


Random projection and all

In [6]:
# import numpy as np
# from sklearn.decomposition import PCA

# # Define z-score normalization function
# def z_score_normalize(data):
#     mean_val = np.mean(data)
#     std_dev = np.std(data)
#     normalized_data = (data - mean_val) / std_dev
#     return normalized_data

# # Define function to apply median filter
# def median_filter(data):
#     # Apply median filter to the entire array
#     filtered_data = np.median(data)
#     print(filtered_data)
#     return filtered_data

# # Define function to apply random projection
# def random_projection(data, n_components):
#     # For 1D arrays, random projection doesn't make sense, but you can just return the original array
#     return data

# # Define function to convert normalized array to binary array
# def to_binary_array(data, threshold):
#     return (data >= threshold).astype(int)

# # Choose normalization method and threshold
# normalize_method = z_score_normalize  # Choose either min_max_scale or z_score_normalize
# threshold = 0.3

# # List to store binary arrays
# binary_arrays = []

# # Normalize, filter, and project each array
# for array in texture_embeddings._array:
#     # Normalize data
#     normalized_array = normalize_method(array)
    
#     # Apply median filter
#     filtered_array = median_filter(normalized_array)
    
#     # # Apply random projection
#     # projected_array = random_projection(filtered_array, n_components=10)  # Adjust n_components as needed
    
#     # Convert to binary array
#     binary_array = to_binary_array(filtered_array, threshold)
    
#     # Append to list
#     binary_arrays.append(binary_array)

# # Print binary arrays
# for i, binary_array in enumerate(binary_arrays):
#     print(f"Binary array for array {i}:")
#     print(binary_array)


Hardcored Core points

In [7]:
# import numpy as np

# # Define z-score normalization function
# def z_score_normalize(data):
#     mean_val = np.mean(data)
#     std_dev = np.std(data)
#     normalized_data = (data - mean_val) / std_dev
#     return normalized_data

# # Define function to convert normalized array to binary array
# def to_binary_array(data, threshold):
#     return (data >= threshold).astype(int)

# # Choose normalization method and threshold
# normalize_method = z_score_normalize  # Choose either min_max_scale or z_score_normalize
# threshold = 0.5

# # List to store binary arrays
# binary_arrays = []

# # Normalize and convert each array to binary
# for array in texture_embeddings._array:
#     # Sort the array (not sure if this step is necessary based on your code)
#     sorted(array)

#     # Normalize the array
#     normalized_array = normalize_method(array)
    
#     # Convert the normalized array to binary using the threshold
#     binary_array = to_binary_array(normalized_array, threshold)
    
#     # Append the binary array to the list
#     binary_arrays.append(binary_array)

# # Generate the permutation pattern from core points (assuming core_points is available)
# core_points = [0, 1, 0, 1]  # Example core points as a binary array

# # Function to permute binary array in intervals using a pattern
# def permute_binary_array_in_intervals(binary_array, pattern):
#     permuted_array = []
#     interval_length = len(binary_array) // len(pattern)
#     for i in range(0, len(binary_array), interval_length):
#         segment = binary_array[i:i+interval_length]
#         permuted_segment = [segment[j] for j in pattern]
#         permuted_array.extend(permuted_segment)
#     return np.array(permuted_array)

# # Permute all binary arrays using the generated pattern
# permuted_arrays = []
# for binary_array in binary_arrays:
#     permuted_binary_array = permute_binary_array_in_intervals(binary_array, core_points)
#     permuted_arrays.append(permuted_binary_array)


# # Print the permuted binary arrays
# for i, permuted_binary_array in enumerate(permuted_arrays):
#     print(f"Permuted binary array for array {i}:")
#     print(permuted_binary_array)


In [11]:
print(embeddings._array)


[[-0.0067844  -0.01064628  0.00425365 ... -0.00924355 -0.0037603
   0.05010139]
 [-0.0067844  -0.01064628  0.00425365 ... -0.00924355 -0.0037603
   0.05010139]
 [-0.0067844  -0.01064628  0.00425365 ... -0.00924355 -0.0037603
   0.05010139]
 ...
 [-0.0067844  -0.01064628  0.00425365 ... -0.00924355 -0.0037603
   0.05010139]
 [-0.0067844  -0.01064628  0.00425365 ... -0.00924355 -0.0037603
   0.05010139]
 [-0.0067844  -0.01064628  0.00425365 ... -0.00924355 -0.0037603
   0.05010139]]


In [15]:
import os

def convert_to_binary(x, y):
    binary_x = bin(int(x))[2:]
    binary_y = bin(int(y))[2:]
    return binary_x, binary_y

def pad_binary(binary, length):
    return binary.zfill(length)

def xor_binary(binary1, binary2):
    return bin(int(binary1, 2) ^ int(binary2, 2))[2:]

def process_txt_file(txt_file):
    with open(txt_file, 'r') as f:
        lines = f.readlines()
        x1, y1 = map(float, lines[0].split())
        x2, y2 = map(float, lines[1].split())
        
        binary_x1, binary_y1 = convert_to_binary(x1, y1)
        binary_x2, binary_y2 = convert_to_binary(x2, y2)
        
        s1 = binary_x1 + binary_y1
        s2 = binary_x2 + binary_y2
        
        max_length = max(len(s1), len(s2))
        s1 = pad_binary(s1, max_length)
        s2 = pad_binary(s2, max_length)
        
        result = xor_binary(s1, s2)
        
        output_file = os.path.splitext(txt_file)[0] + '_result.txt'
        with open(output_file, 'w') as out_f:
            out_f.write(result)

def process_folder(input_folder):
    for filename in os.listdir(input_folder):
        if filename.endswith('.txt'):
            process_txt_file(os.path.join(input_folder, filename))

# Specify input folder containing .txt files
input_folder = '/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/fixedlength-core_points2'

# Process all .txt files in the folder
process_folder(input_folder)


ValueError: not enough values to unpack (expected 2, got 1)

This is the permutation with core as binary value seed

In [12]:
import numpy as np
import os

# Define z-score normalization function
def z_score_normalize(data):
    mean_val = np.mean(data)
    std_dev = np.std(data)
    normalized_data = (data - mean_val) / std_dev
    return normalized_data

# Define function to convert normalized array to binary array
def to_binary_array(data, threshold):
    return (data >= threshold).astype(int)

# Choose normalization method and threshold
normalize_method = z_score_normalize  # Choose either min_max_scale or z_score_normalize
threshold = 0.3

# List to store binary arrays
binary_arrays = []

# Normalize and convert each array to binary
for array in embeddings._array:
    # Sort the array (not sure if this step is necessary based on your code)
    sorted(array)

    # Normalize the array
    normalized_array = normalize_method(array)
    
    # Convert the normalized array to binary using the threshold
    binary_array = to_binary_array(normalized_array, threshold)
    
    # Append the binary array to the list
    binary_arrays.append(binary_array)

# Folder containing core point files
core_points_folder = "/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/walking_binary_core"

# Permute all binary arrays using the core points from files in the folder
permuted_array = []

for binary_array in binary_arrays:
    for core_points_file in sorted(os.listdir(core_points_folder)):
    # Read core points from the current file
        with open(os.path.join(core_points_folder, core_points_file), "r") as f:
            core_points_str = f.read().strip()


    # Convert the core points string to a list of integers
        core_points = [int(bit) for bit in core_points_str]

    # Function to permute binary array in intervals using the pattern
    def permute_binary_array_in_intervals(binary_array, pattern):
        permuted_array = []
        interval_length = len(binary_array) // len(pattern)
        for i in range(0, len(binary_array), interval_length):
            segment = binary_array[i:i+interval_length]
            permuted_segment = [segment[j] for j in pattern]
            permuted_array.extend(permuted_segment)
        return np.array(permuted_array)

    # Permute the binary array using the read core points
    permuted_binary_array = permute_binary_array_in_intervals(binary_array, core_points)
    permuted_array.append(permuted_binary_array)

# Print the permuted binary arrays
for i, permuted_binary_array in enumerate(permuted_array):
    print(f"Permuted binary array for array {i}:")
    print(permuted_binary_array)


Permuted binary array for array 0:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0
 1 1 0 1 1 0 1 0 1 0 1 0 0 1 1 1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 0 1 1 1 0 0 1 0 0 1 0 1 0 1
 0 1 1 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 0 0 1 0 0 1 0 1 0 1 0 0 0 1 1 1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Permuted binary array for array 1:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0
 1 1 0 1 1 0 1 0 1 0 1 0 0 1 1 1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 0 1 1 1 0 0 1 0 0 1 0 1 0 1
 0 1 1 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 0 0 1 0 0 1 0 1 0 1 0 0 0 1 1 1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
# import numpy as np

# # Define z-score normalization function
# def z_score_normalize(data):
#     mean_val = np.mean(data)
#     std_dev = np.std(data)
#     normalized_data = (data - mean_val) / std_dev
#     return normalized_data

# # Define function to convert normalized array to binary array
# def to_binary_array(data, threshold):
#     return (data >= threshold).astype(int)

# # Choose normalization method and threshold
# normalize_method = z_score_normalize  # Choose either min_max_scale or z_score_normalize
# threshold = 0.5

# # List to store binary arrays
# binary_arrays = []

# # Normalize and convert each array to binary
# for array in texture_embeddings._array:
#     # Sort the array (not sure if this step is necessary based on your code)
#     sorted(array)

#     # Normalize the array
#     normalized_array = normalize_method(array)
    
#     # Convert the normalized array to binary using the threshold
#     binary_array = to_binary_array(normalized_array, threshold)
    
#     # Append the binary array to the list
#     binary_arrays.append(binary_array)

# # Generate the permutation pattern from core points (assuming core_points is available)
# core_points = [0, 1, 0, 1]  # Example core points as a binary array

# # Function to permute binary array in intervals using a pattern
# def permute_binary_array_in_intervals(binary_array, pattern):
#     permuted_array = []
#     interval_length = len(binary_array) // len(pattern)
#     for i in range(0, len(binary_array), interval_length):
#         segment = binary_array[i:i+interval_length]
#         permuted_segment = [segment[j] for j in pattern]
#         permuted_array.extend(permuted_segment)
#     return np.array(permuted_array)

# # Permute all binary arrays using the generated pattern
# permuted_arrays = []
# for binary_array in binary_arrays:
#     permuted_binary_array = permute_binary_array_in_intervals(binary_array, core_points)
#     permuted_arrays.append(permuted_binary_array)


# # Print the permuted binary arrays
# for i, permuted_binary_array in enumerate(permuted_arrays):
#     print(f"Permuted binary array for array {i}:")
#     print(permuted_binary_array)


In [ ]:
import numpy as np
import os

# Define z-score normalization function
def z_score_normalize(data):
    mean_val = np.mean(data)
    std_dev = np.std(data)
    normalized_data = (data - mean_val) / std_dev
    return normalized_data

# Define function to convert normalized array to binary array
def to_binary_array(data, threshold):
    return (data >= threshold).astype(int)

# Choose normalization method and threshold
normalize_method = z_score_normalize  # Choose either min_max_scale or z_score_normalize
threshold = 0.3

# List to store binary arrays
binary_arrays = []

# Normalize and convert each array to binary
for array in texture_embeddings._array:
    # Sort the array (not sure if this step is necessary based on your code)
    sorted(array)

    # Normalize the array
    normalized_array = normalize_method(array)
    
    # Convert the normalized array to binary using the threshold
    binary_array = to_binary_array(normalized_array, threshold)
    
    # Append the binary array to the list
    binary_arrays.append(binary_array)

# Folder containing core point files
core_points_folder = "/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/walking_binary_core"

# Permute all binary arrays using the core points from files in the folder
permuted_array = []

for binary_array in binary_arrays:
    for core_points_file in sorted(os.listdir(core_points_folder)):
    # Read core points from the current file
        with open(os.path.join(core_points_folder, core_points_file), "r") as f:
            core_points_str = f.read().strip()


    # Convert the core points string to a list of integers
        core_points = [int(bit) for bit in core_points_str]

    # Function to permute binary array in intervals using the pattern
    def permute_binary_array_in_intervals(binary_array, pattern):
        permuted_array = []
        interval_length = len(binary_array) // len(pattern)
        for i in range(0, len(binary_array), interval_length):
            segment = binary_array[i:i+interval_length]
            permuted_segment = [segment[j] for j in pattern]
            permuted_array.extend(permuted_segment)
        return np.array(permuted_array)

    # Permute the binary array using the read core points
    permuted_binary_array = permute_binary_array_in_intervals(binary_array, core_points)
    permuted_array.append(permuted_binary_array)

# Print the permuted binary arrays
for i, permuted_binary_array in enumerate(permuted_array):
    print(f"Permuted binary array for array {i}:")
    print(permuted_binary_array)


This is for tensors to delta distance and normalise and with threshold get binary array

In [ ]:
# Function to calculate Hamming distance between two binary arrays
def hamming_distance(array1, array2):
    return np.sum(array1 != array2)

# Initialize lists to store binary arrays for each person
binary_arrays_same_person = []
binary_arrays_diff_person = []



# Populate binary arrays lists
for i, permuted_binary_array in enumerate(permuted_array):
    # Append to appropriate list based on person index
    person_index = i // 10  # Assuming each person has 10 impressions
    if person_index < 6:
        binary_arrays_same_person.append(permuted_binary_array)
    else:
        binary_arrays_diff_person.append(permuted_binary_array)

# Calculate Hamming distance for same person pairs
print("Hamming distances for same person pairs:")
for i in range(len(binary_arrays_same_person)):
    for j in range(i + 1, len(binary_arrays_same_person)):
        distance = hamming_distance(binary_arrays_same_person[i], binary_arrays_same_person[j])
        print(f"Impression {i} vs Impression {j}: {distance}")

# Calculate Hamming distance for different person pairs
print("\nHamming distances for different person pairs:")
for i in range(len(binary_arrays_same_person)):
    for j in range(len(binary_arrays_diff_person)):
        distance = hamming_distance(binary_arrays_same_person[i], binary_arrays_diff_person[j])
        print(f"Impression {i} (Same Person) vs Impression {j} (Different Person): {distance}")


In [13]:
import os
import numpy as np

# Function to read delta points from a .txt file
def read_delta_points(delta_file):
    with open(delta_file, 'r') as f:
        lines = f.readlines()
        delta_points = [[float(coord) for coord in line.strip().split()] for line in lines]
    return np.array(delta_points)

# Function to compute Euclidean distance between a tensor and delta points
def compute_distances(tensor, delta_points):
    distances = []
    for delta_point in delta_points:
        distance = np.linalg.norm(tensor[:2] - delta_point)
        distances.append(distance)
    return distances

# Define z-score normalization function
def z_score_normalize(data):
    mean_val = np.mean(data)
    std_dev = np.std(data)
    normalized_data = (data - mean_val) / std_dev
    return normalized_data

# Define function to convert normalized array to binary array
def to_binary_array(data, threshold):
    return (data >= threshold).astype(int)

# Choose normalization method and threshold
normalize_method = z_score_normalize  # Choose either min_max_scale or z_score_normalize
threshold = 0.3

# List to store binary arrays
binary_arrays = []

# Folder containing delta point .txt files
delta_folder = "/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/walking_delta_points"

# Iterate over each .txt file in the folder
for filename in os.listdir(delta_folder):
    if filename.endswith('.txt'):
        # Read delta points from the current file
        delta_file = os.path.join(delta_folder, filename)
        delta_points = read_delta_points(delta_file)

        # Compute distances for each tensor and delta points
        for tensor in embeddings._array:
            distances = compute_distances(tensor, delta_points)

            # Normalize and convert each tensor array to binary
            normalized_array = normalize_method(tensor)
            binary_array = to_binary_array(normalized_array, threshold)
            binary_arrays.append(binary_array)

            # Print binary array
            print(f"Binary array for tensor with distances {distances}:")
            print(binary_array)



Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor with distances []:
[0 0 0 ... 0 0 1]
Binary array for tensor 

In [10]:

# import numpy as np

# # Define z-score normalization function
# def z_score_normalize(data):
#     mean_val = np.mean(data)
#     std_dev = np.std(data)
#     normalized_data = (data - mean_val) / std_dev
#     return normalized_data

# # Define function to convert normalized array to binary array
# def to_binary_array(data, threshold):
#     return (data >= threshold).astype(int)

# # Choose normalization method and threshold
# normalize_method = z_score_normalize  # Choose either min_max_scale or z_score_normalize
# threshold = 0.3

# # List to store binary arrays
# binary_arrays = []

# # Normalize and convert each array to binary
# for array in embeddings._array:
#     sorted(array)

#     normalized_array = normalize_method(array)
#     binary_array = to_binary_array(normalized_array, threshold)
#     binary_arrays.append(binary_array)

# # Print binary arrays
# for i, binary_array in enumerate(binary_arrays):
#     print(f"Binary array for array {i}:")
#     print(binary_array)

# import numpy as np

# # Function to calculate Hamming distance between two binary arrays
# def hamming_distance(array1, array2):
#     return np.sum(array1 != array2)

# # Initialize lists to store binary arrays for each person
# binary_arrays_same_person = []
# binary_arrays_diff_person = []

# # Normalize and convert each array to binary
# for array in minutia_embeddings._array:
#     # Truncate the array to the first 8 bits
#     truncated_array = array[:8]
    
#     # Convert the truncated array to binary
#     binary_array = to_binary_array(truncated_array, threshold)
    
#     # Append to appropriate list based on person index
#     person_index = i % 10  # Assuming each person has 10 impressions
#     if person_index < 10:
#         binary_arrays_same_person.append(binary_array)
#     else:
#         binary_arrays_diff_person.append(binary_array)

# # Calculate Hamming distance for same person pairs
# print("Hamming distances for same person pairs:")
# for i in range(len(binary_arrays_same_person)):
#     for j in range(i + 1, len(binary_arrays_same_person)):
#         distance = hamming_distance(binary_arrays_same_person[i], binary_arrays_same_person[j])
#         print(f"Impression {i} vs Impression {j}: {distance}")

# # Calculate Hamming distance for different person pairs
# print("\nHamming distances for different person pairs:")
# for i in range(len(binary_arrays_diff_person)):
#     for j in range(len(binary_arrays_diff_person)):
#         distance = hamming_distance(binary_arrays_diff_person[i], binary_arrays_diff_person[j])
#         print(f"Impression {i} vs Impression {j}: {distance}")

For permutation one

In [14]:
# Function to calculate Hamming distance between two binary arrays
def hamming_distance(array1, array2):
    return np.sum(array1 != array2)

# Initialize lists to store binary arrays for each person
binary_arrays_same_person = []
binary_arrays_diff_person = []



# Populate binary arrays lists
for i, permuted_binary_array in enumerate(permuted_array):
    # Append to appropriate list based on person index
    person_index = i // 10  # Assuming each person has 10 impressions
    if person_index < 6:
        binary_arrays_same_person.append(permuted_binary_array)
    else:
        binary_arrays_diff_person.append(permuted_binary_array)

# Calculate Hamming distance for same person pairs
print("Hamming distances for same person pairs:")
for i in range(len(binary_arrays_same_person)):
    for j in range(i + 1, len(binary_arrays_same_person)):
        distance = hamming_distance(binary_arrays_same_person[i], binary_arrays_same_person[j])
        print(f"Impression {i} vs Impression {j}: {distance}")

# Calculate Hamming distance for different person pairs
print("\nHamming distances for different person pairs:")
for i in range(len(binary_arrays_same_person)):
    for j in range(len(binary_arrays_diff_person)):
        distance = hamming_distance(binary_arrays_same_person[i], binary_arrays_diff_person[j])
        print(f"Impression {i} (Same Person) vs Impression {j} (Different Person): {distance}")


Hamming distances for same person pairs:
Impression 0 vs Impression 1: 0
Impression 0 vs Impression 2: 0
Impression 0 vs Impression 3: 0
Impression 0 vs Impression 4: 0
Impression 0 vs Impression 5: 0
Impression 0 vs Impression 6: 0
Impression 0 vs Impression 7: 0
Impression 0 vs Impression 8: 0
Impression 0 vs Impression 9: 0
Impression 0 vs Impression 10: 0
Impression 0 vs Impression 11: 0
Impression 0 vs Impression 12: 0
Impression 0 vs Impression 13: 0
Impression 0 vs Impression 14: 0
Impression 0 vs Impression 15: 0
Impression 0 vs Impression 16: 0
Impression 0 vs Impression 17: 0
Impression 0 vs Impression 18: 0
Impression 0 vs Impression 19: 0
Impression 0 vs Impression 20: 0
Impression 0 vs Impression 21: 0
Impression 0 vs Impression 22: 0
Impression 0 vs Impression 23: 0
Impression 0 vs Impression 24: 0
Impression 0 vs Impression 25: 0
Impression 0 vs Impression 26: 0
Impression 0 vs Impression 27: 0
Impression 0 vs Impression 28: 0
Impression 0 vs Impression 29: 0
Impression 

Hamming distance code


In [9]:
import numpy as np

# Function to calculate Hamming distance between two binary arrays
def hamming_distance(array1, array2):
    return np.sum(array1 != array2)

# Initialize lists to store binary arrays for each person
binary_arrays_same_person = []
binary_arrays_diff_person = []

# Normalize and convert each array to binary
for array in embeddings._array:
    normalized_array = normalize_method(array)
    binary_array = to_binary_array(normalized_array, threshold)
    
    # Append to appropriate list based on person index
    person_index = i % 10  # Assuming each person has 10 impressions
    if person_index < 10:
        binary_arrays_same_person.append(binary_array)
    else:
        binary_arrays_diff_person.append(binary_array)

# Calculate Hamming distance for same person pairs
print("Hamming distances for same person pairs:")
for i in range(len(binary_arrays_same_person)):
    for j in range(i + 1, len(binary_arrays_same_person)):
        distance = hamming_distance(binary_arrays_same_person[i], binary_arrays_same_person[j])
        print(f"Impression {i} vs Impression {j}: {distance}")

# Calculate Hamming distance for different person pairs
print("\nHamming distances for different person pairs:")
for i in range(len(binary_arrays_diff_person)):
    for j in range(len(binary_arrays_diff_person)):
        distance = hamming_distance(binary_arrays_diff_person[i], binary_arrays_diff_person[j])
        print(f"Impression {i} vs Impression {j}: {distance}")


Hamming distances for same person pairs:
Impression 0 vs Impression 1: 0
Impression 0 vs Impression 2: 0
Impression 0 vs Impression 3: 0
Impression 0 vs Impression 4: 0
Impression 0 vs Impression 5: 0
Impression 0 vs Impression 6: 0
Impression 0 vs Impression 7: 0
Impression 0 vs Impression 8: 0
Impression 0 vs Impression 9: 0
Impression 0 vs Impression 10: 0
Impression 0 vs Impression 11: 0
Impression 0 vs Impression 12: 0
Impression 0 vs Impression 13: 0
Impression 0 vs Impression 14: 0
Impression 0 vs Impression 15: 0
Impression 0 vs Impression 16: 0
Impression 0 vs Impression 17: 0
Impression 0 vs Impression 18: 0
Impression 0 vs Impression 19: 0
Impression 0 vs Impression 20: 0
Impression 0 vs Impression 21: 0
Impression 0 vs Impression 22: 0
Impression 0 vs Impression 23: 0
Impression 0 vs Impression 24: 0
Impression 0 vs Impression 25: 0
Impression 0 vs Impression 26: 0
Impression 0 vs Impression 27: 0
Impression 0 vs Impression 28: 0
Impression 0 vs Impression 29: 0
Impression 

In [ ]:
import struct
import numpy as np

def write_to_file(name, data, siz):
    f = open(name, 'wb')
    s = struct.pack('i' * len(siz), *siz)
    f.write(s)
    s = struct.pack('f' * len(data), *data)
    f.write(s)
    f.close()

# Assuming texture_embeddings is a 2D array of shape (60, 512)
texture_embeddings = texture_embeddings_array  # Your 2D array

# Normalize texture_embeddings
texture_embeddings_normalized = texture_embeddings / np.linalg.norm(texture_embeddings, ord=2, axis=1, keepdims=True)

precision = 125

# Round the normalized texture_embeddings for precision
d1 = np.round(texture_embeddings_normalized * precision)
d2 = np.round(texture_embeddings_normalized * precision)

# Save in 1-to-1 format
d1_flattened = d1.flatten()
d2_flattened = d2.flatten()

size1 = [60, 512]  # Assuming 60 one-dimensional arrays of 512 dimensions
size2 = [60, 512]

write_to_file('probe-1-to-1.bin', d2_flattened, size2)
write_to_file('gallery-1-to-1.bin', d1_flattened, size1)

# Save in 1-to-n format
d1_transposed = d1.T.flatten()  # Transpose d1 before flattening
d2_transposed = d2.T.flatten()  # Transpose d2 before flattening

size1 = [512, 60]  # Transpose of size for 1-to-n format
size2 = [512, 60]

write_to_file('probe-1-to-n.bin', d2_transposed, size2)
write_to_file('gallery-1-to-n.bin', d1_transposed, size1)


## Benchmarking

To evaluate the embeddings, we want to run a benchmark on them. For this, we must first specify the type of benchmark, and which comparisons should be run.

In [7]:
from flx.scripts.generate_benchmarks import create_verification_benchmark

NUM_IMPRESSIONS_PER_SUBJECT = 10
benchmark = create_verification_benchmark(
    #subjects=list(range(100, image_dataset.num_subjects + 100)),
    subjects=list(range(image_dataset.num_subjects)),
    impressions_per_subject=list(range(NUM_IMPRESSIONS_PER_SUBJECT))
)

100%|██████████| 6/6 [00:00<00:00, 13287.13it/s]


Now we can run the benchmark. To do this, we must first specify the matcher (in our case cosine similarity of the embeddings)

In [8]:
from flx.benchmarks.matchers import CosineSimilarityMatcher
from flx.data.embedding_loader import EmbeddingLoader

# We concatenate texture and minutia embedding vectors
embeddings = EmbeddingLoader.combine(texture_embeddings, minutia_embeddings)
matcher = CosineSimilarityMatcher(EmbeddingLoader.combine(texture_embeddings, minutia_embeddings))

results = benchmark.run(matcher)

print(f"Equal-Error-Rate: {results.get_equal_error_rate()}")

Created IdentifierSet with 6 subjects and a total of 60 samples.
Created IdentifierSet with 6 subjects and a total of 60 samples.
Created IdentifierSet with 6 subjects and a total of 60 samples.
Created IdentifierSet with 6 subjects and a total of 60 samples.
Created IdentifierSet with 6 subjects and a total of 60 samples.
Created IdentifierSet with 6 subjects and a total of 60 samples.


  0%|          | 0/870 [00:00<?, ?it/s]


KeyError: <flx.data.dataset.Identifier object at 0x7f8a938447d0>

To visualize the results, we can plot a DET curve. (Do not wonder if it is empty, probably the model is not trained enough. Take a look at the EER instead.)

In [ ]:
from flx.visualization.plot_DET_curve import plot_verification_results

figure_path = "DET_curve"

# Lists are used to allow for multiple models to be plotted in the same figure
plot_verification_results(figure_path, results=[results], model_labels=["DeepPrint_TexMinu"], plot_title="example-dataset - verification")

/home/mt0/22CS60R42/.anaconda3/lib/python3.11/site-packages/matplotlib/pyplot.py:2201: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if fignum is False or fignum is 0:
/home/mt0/22CS60R42/.anaconda3/lib/python3.11/site-packages/matplotlib/pyplot.py:2201: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if fignum is False or fignum is 0:
/home/mt0/22CS60R42/.anaconda3/lib/python3.11/site-packages/matplotlib/pyplot.py:2201: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if fignum is False or fignum is 0:
/home/mt0/22CS60R42/.anaconda3/lib/python3.11/site-packages/matplotlib/pyplot.py:2201: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if fignum is False or fignum is 0:


SystemError: PY_SSIZE_T_CLEAN macro must be defined for '#' formats

SystemError: PY_SSIZE_T_CLEAN macro must be defined for '#' formats

<Figure size 640x480 with 1 Axes>